In [1]:
import json

import numpy as np
import pandas as pd

import andes
import ams

In [2]:
!andes prep -i


    _           _         | Version 1.9.3.post32+gddc308cb
   /_\  _ _  __| |___ ___ | Python 3.12.0 on Darwin, 05/27/2025 10:02:03 PM
  / _ \| ' \/ _` / -_|_-< | 
 /_/ \_\_||_\__,_\___/__/ | This program comes with ABSOLUTELY NO WARRANTY.

> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
Numerical code generation (rapid incremental mode) started...
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Generating code for 1 models on 12 processes.
Saved generated pycode to "/Users/jinningwang/.andes/pycode"
> Reloaded generated Python code of module "pycode".
Generated numerical code for 1 models in 0.0917 seconds.


In [3]:
andes.config_logger(stream_level=50)
ams.config_logger(stream_level=50)

In [4]:
case_path = "./../cases"
res_path = "./../results"
addfile = case_path + '/IL200_dyn_db2.xlsx'

# --- file loading ---
curve = pd.read_csv(case_path + '/CurveInterp.csv')

In [5]:
sp = ams.load(case_path + '/IL200_opf2.xlsx',
              setup=True, no_output=True,
              default_config=True)

sa = sp.to_andes(addfile=addfile, setup=True, no_output=True,
                 default_config=True)

# new link table
link = sp.dyn.link.copy().fillna(False)
# existence of each type of generator
link['has_gov'] = link['gov_idx'].fillna(
    0, inplace=False).astype(bool).astype(int)
link['has_dg'] = link['dg_idx'].fillna(
    0, inplace=False).astype(bool).astype(int)
link['has_rg'] = link['rg_idx'].fillna(
    0, inplace=False).astype(bool).astype(int)

Generating code for 1 models on 12 processes.


/var/folders/__/n5kx_m_s0tbg6n5qd7rh51700000gn/T/ipykernel_41285/1130107227.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  link = sp.dyn.link.copy().fillna(False)


In [6]:
with open(res_path + f'/opf.json', 'r') as f:
    opf = json.load(f)

In [7]:
# NOTE: 1) the maximum number of dispatch should follow: "D * Dispatch_interval <= 3600"
#       2) the maximum number of AGC should follow: "Dispatch_interval % AGC_interval == 0"

Dispatch_interval = 900  # seconds
AGC_interval = 15  # seconds

pq_idx = sp.PQ.idx.v
p0 = sp.PQ.p0.v.copy()
q0 = sp.PQ.q0.v.copy()

stg = sp.StaticGen.get_all_idxes()

stg_idxes = sp.StaticGen.find_idx(keys='gentype',
                                  values=['W2', 'PV', 'ES'],
                                  allow_all=True)

stg_w2t, stg_pv, stg_ess = stg_idxes

p0_w2t = sp.StaticGen.get(src='p0', attr='v', idx=stg_w2t)
p0_pv = sp.StaticGen.get(src='p0', attr='v', idx=stg_pv)

stg_slack = sp.Slack.idx.v

In [8]:
KP = 0.2
KI = 0.05
ACE_integral = 0.0  # Integral of Area Control Error (ACE)
ACE_raw = 0.0  # Raw Area Control Error (ACE)

In [9]:
for hour in [22]:
    for dispatch in [1]:
        r0 = hour * 3600 + dispatch * Dispatch_interval
        # --- Dispatch ---
        opfr = opf[f"h{hour}d{dispatch}"]

        sa = andes.load(addfile,
                        setup=True, no_output=True,
                        default_config=True)

        syn_slack = sa.SynGen.find_idx(keys='gen', values=stg_slack)[0]

        sap0 = sa.PQ.p0.v.copy()  # Copy of initial ANDES active load
        saq0 = sa.PQ.q0.v.copy()  # Copy of initial ANDES reactive load

        sa.StaticGen.set(src='p0', attr='v', idx=opfr['gen'], value=opfr['pg'])
        sa.Bus.set(src='v0', attr='v', idx=opfr['bus'], value=opfr['vBus'])
        sa.Bus.set(src='a0', attr='v', idx=opfr['bus'], value=opfr['aBus'])

        pv_bus = sa.PV.bus.v
        slack_bus = sa.Slack.bus.v
        v_pv = sa.Bus.get(src='v0', attr='v', idx=pv_bus)
        a_slack = sa.Bus.get(src='a0', attr='v', idx=slack_bus)

        sa.PV.set(src='v0', attr='v', idx=sa.PV.idx.v, value=v_pv)
        sa.Slack.set(src='a0', attr='v', idx=sa.Slack.idx.v, value=a_slack)

        # --- participation factor ---
        stg_on_uid = np.where(np.array(opfr['pg']) > 1e-4)[0]
        stg = opfr['gen']
        stg_on = np.array(
            [1 if uid in stg_on_uid else 0 for uid in range(len(stg))])
        sn = sp.StaticGen.get(src='Sn', attr='v', idx=stg)
        bf = stg_on * sn / (stg_on * sn).sum()

        # use constant power model for PQ
        sa.PQ.config.p2p = 1
        sa.PQ.config.q2q = 1
        sa.PQ.config.p2z = 0
        sa.PQ.config.q2z = 0
        sa.PQ.pq2z = 0

        sa.TDS.config.criteria = 0
        sa.TDS.config.no_tqdm = True

        sa.PQ.set(src='p0', attr='v', idx=pq_idx,
                  value=opfr['load'] * sap0)
        sa.PQ.set(src='q0', attr='v', idx=pq_idx,
                  value=opfr['load'] * saq0)
        
        sa.StaticGen.set(src='p0', attr='v', idx=stg_w2t,
                         value=opfr['wind'] * p0_w2t)
        sa.StaticGen.set(src='p0', attr='v', idx=stg_pv,
                         value=opfr['solar'] * p0_pv)
        sa.PFlow.run()
        _ = sa.TDS.init()

        for t in range(Dispatch_interval):
            # --- AGC Interval ---
            # Compute AGC signals only for generators with corresponding controllers
            for col, has_col in [('agov', 'has_gov'), ('adg', 'has_dg'), ('arg', 'has_rg')]:
                link[col] = ACE_raw * bf * link[has_col] * link['gammap']
            if t % AGC_interval == 0 and t > 0:
                # set into governor, Exclude NaN values for governor index
                agov_to_set = {gov: agov for gov, agov in zip(
                    link['gov_idx'], link['agov']) if bool(gov)}
                sa.TurbineGov.set(src='paux0', idx=list(
                    agov_to_set.keys()), attr='v', value=list(agov_to_set.values()))

                # set into dg, Exclude NaN values for dg index
                adg_to_set = {dg: adg for dg, adg in zip(
                    link['dg_idx'], link['adg']) if bool(dg)}
                sa.DG.set(src='Pext0', idx=list(adg_to_set.keys()),
                            attr='v', value=list(adg_to_set.values()))

            # --- TDS Interval ---
            if t > 0:  # when t>0, run TDS
                # 1) Update loads
                kload = curve['Load'].iloc[r0+t]
                sa.PQ.set(src='Ppf', attr='v', idx=sa.PQ.idx.v,
                            value=kload * sap0)
                sa.PQ.set(src='Qpf', attr='v', idx=sa.PQ.idx.v,
                            value=kload * saq0)
                # ANDES wind and solar
                wind = curve['Wind'].iloc[r0+t]
                sa.StaticGen.set(src='p0', attr='v', idx=stg_w2t,
                                    value=wind * p0_w2t)
                solar = curve['PV'].iloc[r0+t]
                sa.StaticGen.set(src='p0', attr='v', idx=stg_pv,
                                    value=solar * p0_pv)

                sa.TDS.config.tf = t
                sa.TDS.run()

                # 3) Update AGC PI controller
                ACE_raw = -(KP * sa.ACEc.ace.v.sum() +
                            KI * ACE_integral)
                ACE_integral = ACE_integral + sa.ACEc.ace.v.sum()

            if sa.exit_code != 0:
                print(f'ANDES exited with {sa.exit_code} at {t}s')
                exit()

            # --- Watchdog ---
            if t % 100 == 0 and t > 0:
                print(f'Watchdog: Second: {t}')

        # export
        sa.TDS.plt.export_csv(res_path + f'/tds_h{hour}_d{dispatch}.csv')

Generating code for 1 models on 12 processes.


/Users/jinningwang/work/andes/andes/core/model/model.py:790: ComplexWarning: Casting complex values to real discards the imaginary part
  instance.v = np.array(func(*self.s_args[name]),
/Users/jinningwang/work/andes/andes/core/model/model.py:829: ComplexWarning: Casting complex values to real discards the imaginary part
  instance.v[:] = func(*self.s_args[name])
/Users/jinningwang/work/andes/andes/core/model/model.py:829: ComplexWarning: Casting complex values to real discards the imaginary part
  instance.v[:] = func(*self.s_args[name])


Watchdog: Second: 100
Watchdog: Second: 200
Watchdog: Second: 300
Watchdog: Second: 400
Watchdog: Second: 500
Watchdog: Second: 600
Watchdog: Second: 700
Watchdog: Second: 800
